# Named Entity Processing with Impresso Models

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/annotate/NE-processing_ImpressoHF.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## What is this notebook about?

This notebook demonstrates how to use Impresso models for named entity recognition (NER) and entity linking (EL).

NER detects and classifies entities such as persons, locations, and organizations in text, while EL connects recognized entities to unique identifiers in a knowledge base, like Wikipedia or its data counterpart, Wikidata.

In this notebook, both NER and EL are performed using models trained by Impresso and hosted on [Hugging Face](https://huggingface.co/impresso-project/) (thus the 'HF' suffix in the notebook name):

- The **Impresso NER model** is a Transformer model trained on the Impresso HIPE-2020 portion of the [HIPE-2022 dataset](https://github.com/hipe-eval/HIPE-2022-data). It recognizes entity types such as person, location, and organization while supporting the complete [HIPE typology](https://github.com/hipe-eval/HIPE-2022-data/blob/main/documentation/README-hipe2020.md), including coarse and fine-grained entity types as well as components like names, titles, and roles. Additionally, the NER model's backbone ([dbmdz/bert-medium-historic-multilingual-cased](https://huggingface.co/dbmdz/bert-medium-historic-multilingual-cased)) was trained on various European historical datasets, giving it a broader language capability.  This training included data from the Europeana and British Library collections across multiple languages: German, French, English, Finnish, and Swedish. Due to this multilingual backbone, the NER model may also recognize entities in other languages beyond French and German.
  
- The **Impresso NEL model** links detected entities to unique identifiers in Wikipedia and Wikidata or assigns a 'NIL' label (indicating "not in list" in NLP) if no reference is found. The NEL model was trained on various historical datasets (AjMC, CLEF-HIPE-2020, LeTemps, Living with Machines, NewsEye, SoNAR) across multiple languages, including German, French, English, Finnish, and Swedish, to support comprehensive entity linking (EL) and named entity recognition (NER). Its backbone, [mGENRE,](https://huggingface.co/facebook/mgenre-wiki) uses a multilingual text generation approach for Wikipedia entity prediction, trained on 105 languages from Wikipedia.

Both models can also be tested interactively in Hugging Face spaces: the [NER space](https://huggingface.co/spaces/impresso-project/multilingual-named-entity-recognition) and the [EL space](https://huggingface.co/spaces/impresso-project/multilingual-entity-linking).

## What will you learn in this notebook?

By the end of this notebook, you will know how to:
- Install the necessary libraries to run the models
- Load the models and modules from Hugging Face
- Perform NER on a text input
- Perform EL on the NER output

**Warning**:
To use this notebook, you may need to set the `HF_TOKEN` environment variable in the `.env` file (refer to `.env.example`). You can obtain a token by signing up on the [Hugging Face website](https://huggingface.co/join) and find additional information in the [official documentation](https://huggingface.co/docs/huggingface_hub/v0.20.2/en/quick-start#environment-variable). If you do not want to register an account on HF, simply select Cancel when prompted for a Hugging Face token — no token is needed for this notebook.

## Prerequisites
First, we install and download necessary libraries:

- **torch**: PyTorch is a popular open-source library for deep learning that provides tools for tensor computation, GPU acceleration, and building neural networks.
- **protobuf**: Protobuf, short for 'Protocol Buffers', is a library developed by Google for serializing structured data in a fast and efficient way, ideal for communication between services. 
- **sentencepiece**: SentencePiece is a text processing library used primarily for tokenization, especially for languages with complex scripts. It supports subword tokenization, which is key for training language models that need flexible token units (e.g., parts of words). BERT and transformers in general often use SentencePiece for multilingual support.
- **transformers**:  Developed by Hugging Face, this library offers many functionalities to support the development of NLP deep learning models. It provides pre-trained models for various tasks, supports architectures like BERT, GPT, T5, and others for model developement and manipulation, offers useful pipelines and easily integrates with PyTorch. The models developed by Impresso are BERT-based-
**nltk**: The Natural Language Toolkit is a library for NLP in Python that offers tools for text processing tasks like tokenization, stemming, lemmatization, and parsing, as well as datasets for linguistic research and training. 

Libraries can be installed from the notebook, or within your environment:

```bash
pip install torch protobuf sentencepiece transformers nltk
```

In [ ]:
!pip install torch
!pip install protobuf
!pip install sentencepiece
!pip install transformers
!pip install nltk

## Entity Recognition

In [ ]:
# Import necessary Python modules from the Transformers library
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline

For NER, we use the Impresso NER model named 'ner-stacked-bert-multilingual' and published on Hugging Face: https://huggingface.co/impresso-project/ner-stacked-bert-multilingual.

In [ ]:
# We set the model_name variable to our chosen model, enabling us to load it and use it for token classification and NER
MODEL_NAME = "impresso-project/ner-stacked-bert-multilingual"

# Load the tokenizer corresponding to the model
ner_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

It is necessary to create a pipeline for our task (`generic-ner`), using the loaded model and tokenizer. This pipeline handles multiple tasks under the hood. This custom NER pipeline streamlines tokenization, language-specific rules, and post-processing into a single workflow. It accurately identifies, aligns, and cleans entities while managing complexities like multilingual punctuation rules, attachment of complementary information (e.g., titles), and removal of redundant tokens. Using this pipeline simplifies our task by handling the NER within a single, streamlined process, making the workflow efficient and minimizing manual data processing.


In [ ]:
ner_pipeline = pipeline("generic-ner", model=MODEL_NAME, 
                        tokenizer=ner_tokenizer, 
                        trust_remote_code=True,
                        device='cpu')

In [ ]:
# We define some test input
sentence = """In the year 1789, King Louis XVI, ruler of France, convened the Estates-General at the Palace of Versailles, 
                where Marie Antoinette, the Queen of France, alongside Maximilien Robespierre, a leading member of the National Assembly, 
                debated with Jean-Jacques Rousseau, the famous philosopher, and Charles de Talleyrand, the Bishop of Autun, 
                regarding the future of the French monarchy. At the same time, across the Atlantic in Philadelphia, 
                George Washington, the first President of the United States, and Thomas Jefferson, the nation's Secretary of State, 
                were drafting policies for the newly established American government following the signing of the Constitution."""

print(sentence)

In [ ]:
# A function that formats and displays the model output in a readable structure
def print_nicely(data):
    for idx, entry in enumerate(data):
        for key, value in entry.items():
            print(f"  {key.capitalize()}: {value}")
        print()  # Blank line between entries
        

We apply the pipeline on the input and print nicely the output

In [ ]:
# Recognize stacked entities for each sentence
entities = ner_pipeline(sentence)

# Extract coarse and fine entities
print_nicely(entities)

### Example of Entity Recognition with OCR Errors

Below, we introduce simulated OCR errors, such as character misrecognition, missing spaces, and incorrect capitalization.

In [ ]:
sentence_with_ocr_errors = """In the year 1789, K1ng L0uis XVl, ruler of France, convened the Estatzs-General at the Palaceof Versailles,
                where Marie Antoinette, the Qveen of France, alongside Max1milien Robespierre, a leading member of the National Assembly,
                debated with JeanJacques Rousseau, the fam0us philos0pher, and Charles de Talleyrand, the B1shop of Autun,
                regarding the futureoftheFrench monarchy. At the same time, across the Atlant1c in Philadelp1ia,
                GeorgeWashington, the first President of the United States, and Thomas Jeffers0n, the nation’s SecretaryofState,
                were drafting policies for the newly establ1shed American govemment foll0wing the sign1ng of the Const1tution."""


Now, let’s run the OCR-affected text through the NER pipeline to observe how well the algorithm performs under OCR-induced distortions.


In [ ]:
entities_with_errors = ner_pipeline(sentence_with_ocr_errors)

print_nicely(entities_with_errors)

In [ ]:
# Verify that the entity counts match for the original and OCR-affected sentences
original_entities = ner_pipeline(sentence)
entities_with_errors = ner_pipeline(sentence_with_ocr_errors)

print("Number of entities in the original text:", len(original_entities))
print("Number of entities in the OCR-affected text:", len(entities_with_errors))
print("Are entity counts equal?", len(original_entities) == len(entities_with_errors))


## Entity Linking

With the EL model, we can link the previously recognised entity mentions to unique referents in Wikipedia and Wikidata.

We use the Impresso model named 'nel-mgenre-multilingual' and published on Hugging Face: https://huggingface.co/impresso-project/nel-mgenre-multilingual.

In [ ]:
# Import the necessary modules from the transformers library
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

NEL_MODEL_NAME = "impresso-project/nel-mgenre-multilingual"

# Load the tokenizer and model from the specified pre-trained model name
# The model used here is "https://huggingface.co/impresso-project/nel-mgenre-multilingual"
nel_tokenizer = AutoTokenizer.from_pretrained("impresso-project/nel-mgenre-multilingual")

In [ ]:
nel_pipeline = pipeline("generic-nel", model=NEL_MODEL_NAME, 
                        tokenizer=nel_tokenizer, 
                        trust_remote_code=True,
                        device='cpu')

Our entity linker requires a specific format to correctly identify the entity that needs to be linked, as follows:

```
The event was held at the [START] Palace of Versailles [END], a symbol of French monarchy.
```

Assuming that the string "Palace of Versailles" was previously detected by an NER tool, we need to surround it with the `[START]` and `[END]` markers. 

The EL pipeline processes only one entity per input text. Therefore, for multiple entities within the same input, we must create separate inputs for each entity. For instance:

```
The event was held at the [START] Palace of Versailles [END], a symbol of French monarchy.
The event was held at the Palace of Versailles, a symbol of [START] French monarchy [END].
```

Let's take this example:

In [ ]:
simple_sentence = "The event was held at the [START] Palace of Versailles [END], a symbol of French monarchy."

linked_entity = nel_pipeline(simple_sentence)

print_nicely(linked_entity)

It _could_ work without the special markers and texts mentioning only one entity, but we do not recommend it.

In [ ]:
simple_sentence = "The event was held at the Palace of Versailles, a symbol of French monarchy."

linked_entity = nel_pipeline(simple_sentence)

print_nicely(linked_entity)

By using our NER tool, we can automatically generate sentences with entity markers and subsequently link each entity:

In [ ]:
# Run the NER pipeline on the input sentence and store the results
entities = ner_pipeline(sentence)

print(f'{len(entities)} entities were detected.')

# List to keep track of already processed words to avoid duplicate tagging
already_done = []

# Process each entity for linking
for entity in entities:
    if entity['surface'] not in already_done:
        # Tag the entity in the text

        language = 'en'
        tokens = sentence.split(' ')
        start, end = (
            entity["index"][0],
            entity["index"][1],
        )

        context_start = max(0, start - 10)
        context_end = min(len(tokens), end + 11)

        nel_sentence = (
            " ".join(tokens[context_start:start])
            + " [START] "
            + entity['surface']
            + " [END] "
            + " ".join(tokens[end + 1 : context_end])
        )

        linked_entities = nel_pipeline(nel_sentence)
        print(nel_sentence)
        print_nicely(linked_entities)

### Example of Entity Linking with OCR Errors

To evaluate the robustness of entity linking with OCR errors, we use both the original and OCR-affected sentences. Below, the entities identified by NER are linked individually to unique Wikipedia/Wikidata entries, while OCR errors are present.



In [ ]:
print(f'{len(entities_with_errors)} entities were previously detected in OCR-affected text.')

# List to avoid reprocessing the same entities
already_done_ocr = []

# Process each detected entity in OCR-affected text for linking
for entity in entities_with_errors:
    if entity['surface'] not in already_done_ocr:
        # Format sentence with entity markers for EL
        language = 'en'
        tokens = sentence_with_ocr_errors.split(' ')
        start, end = entity["index"][0], entity["index"][1]

        context_start = max(0, start - 10)
        context_end = min(len(tokens), end + 11)

        # Surround entity with [START] and [END] tags
        nel_sentence = (
            " ".join(tokens[context_start:start])
            + " [START] "
            + entity['surface']
            + " [END] "
            + " ".join(tokens[end + 1:context_end])
        )

        # Perform entity linking on OCR-affected sentence
        linked_entity_ocr = nel_pipeline(nel_sentence)
        print("Sentence with OCR Error:")
        print(nel_sentence)
        print("Linked Entity:")
        print_nicely(linked_entity_ocr)
        already_done_ocr.append(entity['surface'])


## Looking up entities in the Impresso Corpus

Are the previously recognised entities present in the Impresso Corpus? For each entity, we use impresso_session.entities.find() to look it up by name. This search will attempt to find a match for the exact name provided. If OCR errors are introduced (e.g., "Max1milien Robespierre" instead of "Maximilien Robespierre"), we can observe how resilient the search function is to variations. Let's explore using the Impresso API and Python Library. 

In [ ]:
from impresso import connect

impresso_session = connect()

In [ ]:
entity = impresso_session.entities.find("Maximilien Robespierre")

entity

This command checks if "Maximilien Robespierre" exists in the Impresso database. Similarly, we test the resilience of the search function by querying slightly altered names (e.g., "Max1milien Robespierre").



In [ ]:
entity = impresso_session.entities.find("Max1milien Robespierre")

entity

In [ ]:
entity = impresso_session.entities.find("Marie Antoinette")

entity